# Wrangling act on Covid19 and Weather Data

In [1]:
import numpy as np
import pandas as pd
import requests
import datetime 
from bs4 import BeautifulSoup

import configparser

## Gathering Data
### US Covid19 Data

In [32]:
us_covid = pd.read_csv('us_covid.csv')
us_covid.head(2)

submission_date state  tot_cases  conf_cases  prob_cases  new_case  \
0      12/01/2021    ND     163565    135705.0     27860.0       589   
1      08/17/2020    MD     100715         NaN         NaN       503   

   pnew_case  tot_death  conf_death  prob_death  new_death  pnew_death  \
0      220.0       1907         NaN         NaN          9         0.0   
1        0.0       3765      3616.0       149.0          3         0.0   

               created_at consent_cases consent_deaths  
0  12/02/2021 02:35:20 PM         Agree      Not agree  
1  08/19/2020 12:00:00 AM           NaN          Agree

### US States Data

In [33]:
us_states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI',
       'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI',
       'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC',
       'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT',
       'VT', 'VA', 'WA', 'WV', 'WI', 'WY']

In [34]:
response = requests.get('https://vanwilson.info/2014/11/sample-zip-codes-50-states/')
soup = BeautifulSoup(response.content, 'lxml')

In [35]:
city = []
state = []
abbr = []
zip_code = []
        
for row in soup.find_all('table')[0].tbody.find_all('tr'):

        col = row.find_all('td')
        if col[2].text in us_states:
            city.append(col[0].text)
            state.append(col[1].text)
            abbr.append(col[2].text)
            zip_code.append(col[3].text)
            
states = pd.DataFrame({'capital_city':city, 'state':abbr, 'state_full_name':state, 'zip_code':zip_code})    

In [36]:
states.head(2)

capital_city state state_full_name zip_code
0   Montgomery    AL         Alabama    36104
1       Juneau    AK          Alaska    99801

### US Weather Data

In [8]:
config = configparser.ConfigParser()
config.read('config.cfg')

api_key = config['WEATHER_BIT']['API_KEY']

In [9]:
weather_url = 'https://api.weatherbit.io/v2.0/history/daily?key={api_key}&postal_code={postal_code}&country=US&start_date={start_date}&end_date={end_date}'
zip_codes = list(states.zip_code.values)

In [10]:
weather_columns = ['state', 'rh', 'max_wind_spd_ts', 't_ghi', 'max_wind_spd', 'solar_rad', 'wind_gust_spd', 
                   'max_temp_ts', 'min_temp_ts', 'clouds', 'max_dni', 'precip_gpm', 'wind_spd', 'slp', 'ts', 
                   'max_ghi', 'temp', 'pres', 'dni', 'dewpt', 'snow', 'dhi', 'precip', 'wind_dir', 'max_dhi', 
                   'ghi', 'max_temp', 't_dni', 'max_uv', 't_dhi', 'datetime', 't_solar_rad', 'min_temp', 
                   'max_wind_dir', 'snow_depth']
weather_data = pd.DataFrame(columns=weather_columns)

In [69]:
# # Downloading data from weatherbit url
# covid_state_date = datetime.date.fromisoformat('2020-01-22')
# covid_end_date = datetime.date.fromisoformat('2022-03-01') + datetime.timedelta(days=1)

# for zip_code in zip_codes[38:]:
#     print(zip_code)
#     start_date = covid_state_date
#     end_date = start_date + datetime.timedelta(days=30)
    
#     while end_date < covid_end_date: 
#         url = weather_url.format(api_key=api_key, 
#                                  postal_code=zip_code, 
#                                  start_date=start_date, 
#                                  end_date=end_date)
        
#         response = requests.get(url)
        
#         if response.status_code == 200:
#             response = response.json()
            
#             for data in response['data']:
#                 data['state'] = response['state_code']
#                 weather_data = weather_data.append(data, ignore_index=True)
                
#         else:
#             raise Exception(response.text)
            
#         start_date = end_date
#         end_date = start_date + datetime.timedelta(days=30)
        
    
#     if start_date < covid_end_date:
#         url = weather_url.format(api_key=api_key, 
#                                  postal_code=zip_code, 
#                                  start_date=start_date, 
#                                  end_date=covid_end_date)

#         response = requests.get(url)

#         if response.status_code == 200:
#             response = response.json()

#             for data in response['data']:
#                 data['state'] = response['state_code']
#                 weather_data = weather_data.append(data, ignore_index=True)
                
#         else:
#             raise Exception(response.text)

In [67]:
weather_data.to_csv('weather_raw.csv', index=False)

In [11]:
# Alternative fetch from local_drive
weather_data = pd.read_csv('weather_raw.csv')

In [14]:
weather_data.head(2)

state    rh  max_wind_spd_ts   t_ghi  max_wind_spd  solar_rad  \
0    AL  77.5       1640026800  3411.4           7.6       32.9   
1    AL  88.0       1640077200  3411.3           5.4       29.4   

   wind_gust_spd  max_temp_ts  min_temp_ts  clouds  ...    ghi  max_temp  \
0            7.6   1640023200   1639987200     100  ...  142.1      10.6   
1            5.4   1640124000   1640142000      89  ...  142.1       9.4   

    t_dni  max_uv  t_dhi    datetime  t_solar_rad  min_temp  max_wind_dir  \
0  6547.6     1.0  776.5  2021-12-20        789.7       2.8         116.0   
1  6544.4     1.0  776.3  2021-12-21        705.5       3.9         154.0   

   snow_depth  
0         NaN  
1         NaN  

[2 rows x 35 columns]

## Accessing Data

### `us_covid` accessing

In [37]:
us_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46140 entries, 0 to 46139
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   submission_date  46140 non-null  object 
 1   state            46140 non-null  object 
 2   tot_cases        46140 non-null  int64  
 3   conf_cases       25104 non-null  float64
 4   prob_cases       25032 non-null  float64
 5   new_case         46140 non-null  int64  
 6   pnew_case        42268 non-null  float64
 7   tot_death        46140 non-null  int64  
 8   conf_death       24760 non-null  float64
 9   prob_death       24760 non-null  float64
 10  new_death        46140 non-null  int64  
 11  pnew_death       42253 non-null  float64
 12  created_at       46140 non-null  object 
 13  consent_cases    38445 non-null  object 
 14  consent_deaths   39219 non-null  object 
dtypes: float64(6), int64(4), object(5)
memory usage: 5.3+ MB


Some datatypes are inconsistent:
* `submission_data` and `created_at`: object instead of datetime
* `conf_cases`, `prob_cases`, `pnew_case`, `conf_death`, `prob_death`, `pnew_death`: float instead of int. This would be because pandas int type has not representation of `np.nan` so the best alternative float type is used.


We can also notice to missing values in `conf_cases`, `prob_cases`, `pnew_case`, `conf_death`, `prob_death`, `pnew_death`, `consent_cases` and `consent_deaths`.



In [38]:
us_covid.describe()

tot_cases    conf_cases     prob_cases       new_case  \
count  4.614000e+04  2.510400e+04   25032.000000   46140.000000   
mean   4.155611e+05  4.347998e+05   60944.166587    1700.025206   
std    7.578479e+05  7.033883e+05   95243.169569    5415.766401   
min    0.000000e+00  0.000000e+00       0.000000  -10199.000000   
25%    7.704500e+03  4.326350e+04      47.750000      28.000000   
50%    1.266070e+05  1.941055e+05   12135.500000     393.000000   
75%    5.163890e+05  6.014852e+05   91516.250000    1477.000000   
max    8.960997e+06  8.381196e+06  595837.000000  319809.000000   

           pnew_case     tot_death    conf_death    prob_death     new_death  \
count   42268.000000  46140.000000  24760.000000  24760.000000  46140.000000   
mean      267.633671   7064.657867   7490.103433    809.153958     20.098006   
std      1363.801315  11574.205227   9533.162955   1213.854667     47.527956   
min   -171804.000000      0.000000      0.000000      0.000000   -352.000000   
25%         0.000000    152.000000    868.000000      0.000000      0.000000   
50%         4.000000   2186.000000   3870.500000    230.000000      4.000000   
75%       178.000000   8930.250000  10467.250000   1094.000000     20.000000   
max     73735.000000  84700.000000  71408.000000   6259.000000   1178.000000   

         pnew_death  
count  42253.000000  
mean       2.053890  
std       27.843081  
min    -2594.000000  
25%        0.000000  
50%        0.000000  
75%        1.000000  
max     2919.000000

We have negative `new_case`, `pnew_case`, `new_death` and `pnew_death`. 

Drilling in a bit on the negative values in the columns above.

In [39]:
us_covid.query('new_case < 0').head(2)

submission_date state  tot_cases  conf_cases  prob_cases  new_case  \
505      06/11/2020    GU        182         NaN         NaN        -1   
600      06/17/2021    NE     223973         NaN         NaN      -157   

     pnew_case  tot_death  conf_death  prob_death  new_death  pnew_death  \
505        0.0          5         NaN         NaN          0         0.0   
600        0.0       2258         NaN         NaN          0         0.0   

                 created_at consent_cases consent_deaths  
505  06/12/2020 02:17:59 PM     Not agree      Not agree  
600  06/17/2021 12:00:00 AM     Not agree      Not agree

> I think we will get a better understanding of this once we set the datatype for `submission_date` that way we are able to order the data and see if the `tot_cases` reduces when ever we have negative `new_case`. 

### `states` accessing

In [18]:
states.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   capital_city     50 non-null     object
 1   state            50 non-null     object
 2   state_full_name  50 non-null     object
 3   zip_code         50 non-null     object
dtypes: object(4)
memory usage: 1.7+ KB


Has all the 50 states, their names and their capital city, I don't see anything wrong with this dataset.

### `weather_data` accessing

In [19]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39465 entries, 0 to 39464
Data columns (total 35 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   state            39465 non-null  object 
 1   rh               39465 non-null  float64
 2   max_wind_spd_ts  39465 non-null  int64  
 3   t_ghi            39465 non-null  float64
 4   max_wind_spd     39465 non-null  float64
 5   solar_rad        39465 non-null  float64
 6   wind_gust_spd    39465 non-null  float64
 7   max_temp_ts      39465 non-null  int64  
 8   min_temp_ts      39465 non-null  int64  
 9   clouds           39465 non-null  int64  
 10  max_dni          39465 non-null  float64
 11  precip_gpm       39465 non-null  float64
 12  wind_spd         39465 non-null  float64
 13  slp              39454 non-null  float64
 14  ts               39465 non-null  int64  
 15  max_ghi          39465 non-null  float64
 16  temp             39465 non-null  float64
 17  pres        

#### Data description 
We would be droping some of the columns to just the few that we really need.
* datetime: Date (YYYY-MM-DD).
* ts: Timestamp UTC (Unix Timestamp).
* pres: Average pressure (mb).
* slp: Average sea level pressure (mb).
* wind_spd: Average wind speed (Default m/s).
* wind_gust_spd: Wind gust speed (m/s).
* max_wind_spd: Maximum 2 minute wind speed (m/s).
* wind_dir: Average wind direction (degrees).
* max_wind_dir: Direction of maximum 2 minute wind gust (degrees).
* max_wind_ts: Time of maximum wind gust UTC (Unix Timestamp).
* temp: Average temperature (default Celcius).
* max_temp: Maximum temperature (default Celcius).
* min_temp: Minimum temperature (default Celcius).
* max_temp_ts: Time of daily maximum temperature UTC (Unix Timestamp).
* min_temp_ts: Time of daily minimum temperature UTC (Unix Timestamp).
* rh: Average relative humidity (%).
* dewpt: Average dew point (default Celcius).
* clouds: [Satellite based] average cloud coverage (%).
* precip: Accumulated precipitation (default mm).
* precip_gpm: Accumulated precipitation [satellite/radar estimated] (default mm).
* snow: Accumulated snowfall (default mm).
* snow_depth: Snow Depth (default mm).
* solar_rad: Average solar radiation (W/M^2)
* t_solar_rad: Total solar radiation (W/M^2)
* ghi: Average global horizontal solar irradiance (W/m^2).
* t_ghi: Day total global horizontal solar irradiance (W/m^2) [Clear Sky]
* max_ghi: Maximum value of global horizontal solar irradiance in day (W/m^2) [Clear Sky]
* dni: Average direct normal solar irradiance (W/m^2) [Clear Sky]
* t_dni: Day total direct normal solar irradiance (W/m^2) [Clear Sky]
* max_dni: Maximum value of direct normal solar irradiance in day (W/m^2) [Clear Sky]
* dhi: Average diffuse horizontal solar irradiance (W/m^2) [Clear Sky]
* t_dhi: Day total diffuse horizontal solar irradiance (W/m^2) [Clear Sky]
* max_dhi: Maximum value of diffuse horizontal solar irradiance in day (W/m^2) [Clear Sky]
* max_uv: Maximum UV Index (0-11+)
* state: Abbreviations of US states

Inconsistent datatype:
* `datetime`: object instead of datetime.

## Cleaning Data

### Inconsistent datatypes (us_covid)
In `us_covid`
* `submission_data` and `created_at`: object instead of datetime
* `conf_cases`, `prob_cases`, `pnew_case`, `conf_death`, `prob_death`, `pnew_death`: float instead of int. This would be because pandas int type has not representation of `np.nan` so the best alternative float type is used.

In [40]:
# Converting datetime columns to datetime
us_covid['submission_date'] = pd.to_datetime(us_covid['submission_date'])
us_covid['created_at'] = pd.to_datetime(us_covid['created_at'])

In [41]:
# We want to drop some columns that we don't need and focus on the few that we do need.
important_columns = ['submission_date', 'state', 'tot_cases', 'conf_cases', 
                     'new_case', 'tot_death', 'conf_death', 'new_death']
us_covid = us_covid[important_columns]

# For the places where we have NaN, we want to fill with 0
us_covid.fillna(0, inplace=True)

# Then we change the datatypes of conf_cases, and conf_death to integer
us_covid['conf_cases'] = us_covid['conf_cases'].astype('int')
us_covid['conf_death'] = us_covid['conf_death'].astype('int')

us_covid.head()

submission_date state  tot_cases  conf_cases  new_case  tot_death  \
0      2021-12-01    ND     163565      135705       589       1907   
1      2020-08-17    MD     100715           0       503       3765   
2      2021-05-31    CA    3685032     3685032       644      62011   
3      2021-07-20    MD     464491           0       155       9822   
4      2020-02-06    NE          0           0         0          0   

   conf_death  new_death  
0           0          9  
1        3616          3  
2       62011          5  
3        9604          3  
4           0          0

In [42]:
us_covid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 46140 entries, 0 to 46139
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   submission_date  46140 non-null  datetime64[ns]
 1   state            46140 non-null  object        
 2   tot_cases        46140 non-null  int64         
 3   conf_cases       46140 non-null  int64         
 4   new_case         46140 non-null  int64         
 5   tot_death        46140 non-null  int64         
 6   conf_death       46140 non-null  int64         
 7   new_death        46140 non-null  int64         
dtypes: datetime64[ns](1), int64(6), object(1)
memory usage: 2.8+ MB


In [49]:
# Get main us states
us_covid = us_covid.query('state in @us_states')

### Too many columns  (weather_data)

In [23]:
# We want to work with just a few of the columns
important_w_columns = ['state', 'datetime', 'pres', 'wind_spd', 'wind_dir', 'temp', 'rh', 'dewpt',
                       'clouds', 'precip', 'snow', 'solar_rad']

weather_data = weather_data[important_w_columns]
weather_data.head()

state    datetime    pres  wind_spd  wind_dir  temp    rh  dewpt  clouds  \
0    AL  2021-12-20  1015.1       3.0     116.0   6.8  77.5    2.6     100   
1    AL  2021-12-21  1008.9       2.8     154.0   8.1  88.0    3.9      89   
2    AL  2021-12-22  1011.1       2.7     288.0   8.0  67.4    1.3       3   
3    AL  2021-12-23  1016.1       2.0     119.0   7.2  68.9    0.7      18   
4    AL  2021-12-24  1013.4       3.1     225.0  10.4  74.7    6.9      22   

   precip  snow  solar_rad  
0     0.0   0.0       32.9  
1    12.5   0.0       29.4  
2     0.0   0.0      142.2  
3     0.0   0.0      142.3  
4     0.0   0.0      126.6

#### Data description 
* `state`: Abbreviations of US states
* `datetime`: Date (YYYY-MM-DD).
* `pres`: Average pressure (mb).
* `wind_spd`: Average wind speed (Default m/s).
* `wind_dir`: Average wind direction (degrees).
* `temp`: Average temperature (default Celcius).
* `rh`: Average relative humidity (%).
* `dewpt`: Average dew point (default Celcius).
* `clouds`: [Satellite based] average cloud coverage (%).
* `precip`: Accumulated precipitation (default mm).
* `snow`: Accumulated snowfall (default mm).
* `solar_rad`: Average solar radiation (W/M^2)

### Inconsistent data type  (weather_data)

In [24]:
weather_data['datetime'] = pd.to_datetime(weather_data['datetime'])

In [25]:
weather_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39465 entries, 0 to 39464
Data columns (total 12 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   state      39465 non-null  object        
 1   datetime   39465 non-null  datetime64[ns]
 2   pres       39447 non-null  float64       
 3   wind_spd   39465 non-null  float64       
 4   wind_dir   39454 non-null  float64       
 5   temp       39465 non-null  float64       
 6   rh         39465 non-null  float64       
 7   dewpt      39465 non-null  float64       
 8   clouds     39465 non-null  int64         
 9   precip     39465 non-null  float64       
 10  snow       39465 non-null  float64       
 11  solar_rad  39465 non-null  float64       
dtypes: datetime64[ns](1), float64(9), int64(1), object(1)
memory usage: 3.6+ MB


## Export cleaned data

In [50]:
# Saving US covid data to csv
us_covid.to_csv('us_covid_cleaned.csv', index=False)

In [27]:
# Saving US states data to csv
states.to_csv('us_states_cleaned.csv', index=False)

In [28]:
# Saving weather data to csv
weather_data.to_csv('weather_cleaned.csv', index=False)

> Now we head on to tableau to find insights and build dashboard

In [53]:
# start_date = datetime.date.fromisoformat('2022-01-01')
# end_date = start_date + datetime.timedelta(days=1)

# for zip_code in zip_codes:
#     print(zip_code)
    
#     url = weather_url.format(api_key=api_key, 
#                              postal_code=zip_code, 
#                              start_date=start_date, 
#                              end_date=end_date)

#     response = requests.get(url)

#     if response.status_code == 200:
#         response = response.json()
#         us_lat_lon[response['state_code']] = {'lat':response['lat'], 'lon':response['lon']}
#     else:
#         raise Exception(response.text)